In [6]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.integrate import quad
import matplotlib.pylab as plt
from math import log, sqrt, exp

In [7]:
def Black76Call(S, K,disc, sigma, T):
    d1 = (np.log(S/K)+sigma**2/2*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return disc*(S*norm.cdf(d1) - K*norm.cdf(d2))

def Black76Put(S, K, disc,sigma, T):
    d1 = (np.log(S/K)+sigma**2/2*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return disc*(K*norm.cdf(-d2) - S*norm.cdf(-d1))

def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*log(F/X)
        zhi = log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [3]:
def IRR(x,N,m):
    IRR=np.zeros(N*m)
    IRRS=0
    for i in range(N*m):
        IRR[i]= 1/m / (1+x/m)**i
    IRRS=np.sum(IRR[:])
    return IRRS

def IRRf(x,N,m):
    dx = 0.05 * x
    IRRplus= IRR(x+dx,N,m)
    IRRminus = IRR (x-dx,N,m)
    IRRf = (IRRplus - IRRminus) / (2*dx)
    return IRRf

def IRRff(x,N,m):
    dx = 0.05 * x
    IRRplus= IRR(x+dx,N,m)
    IRRx = IRR(x,N,m)
    IRRminus = IRR (x-dx,N,m)
    IRRff = (IRRplus - 2*IRRx + IRRminus) / (dx**2)
    return IRRff

def hf(x, N, m):
    term1 = IRR(x, N, m) * (1/4) * x ** (-3 / 4)
    term2 = (x ** (1/4) - 0.2) * IRRf(x, N, m)
    return (term1 - term2) / (IRR(x, N, m)**2)

def hff(x, N, m):
    term1 = IRR(x, N, m) * (-3 / 16 * x ** (-7/4))
    term2 = IRRff(x, N, m) * (x**(1/4) - 0.2)
    term3 = 2 * IRRf(x, N, m) * (1/4) * x ** (-3/4)
    term4 = 2 * IRRf(x, N, m) ** 2 * (x ** (1/4) - 0.2)
    h = (term1 - term2 - term3) / (IRR(x,N,m)**2) + term4 / (IRR(x,N,m)**3)
    return h

def integral1(x,N,m,F,disc,sigma,T):
    h = hff(x, N, m)
    Vrec=Black76Put(F, x, disc, sigma, T)
    return h*Vrec

def integral2(x,N,m,F,disc,sigma,T):
    h = hff(x, N, m)
    Vpay=Black76Call(F, x, disc, sigma, T)
    return h*Vpay

In [4]:
data = pd.read_csv('CMS_10Y.csv', header = 0, index_col = 0)
data

,Start,Tenor,FS,DF,alpha,rho,nu,CMS
0,0.5Y,10Y,0.037845,0.998752,0.171073,-0.264900,0.778199,0.038037
1,1Y,10Y,0.038428,0.997009,0.171073,-0.264900,0.778199,0.038844
2,1.5Y,10Y,0.039020,0.995270,0.171213,-0.280800,0.747216,0.039688
3,2Y,10Y,0.039634,0.993531,0.171354,-0.296700,0.716233,0.040582
4,2.5Y,10Y,0.040200,0.991773,0.171494,-0.312600,0.685251,0.041448
5,3Y,10Y,0.040788,0.990015,0.171635,-0.328500,0.654268,0.042359
6,3.5Y,10Y,0.041412,0.988066,0.171775,-0.344401,0.623285,0.043325
7,4Y,10Y,0.042062,0.986117,0.171916,-0.360301,0.592302,0.044335
8,4.5Y,10Y,0.042831,0.984150,0.172057,-0.376201,0.561320,0.045489
9,5Y,10Y,0.043634,0.982184,0.172197,-0.392101,0.530337,0.046695


In [5]:
F = data.loc[9, 'FS']
D = data.loc[9, 'DF']
alpha = data.loc[9, 'alpha']
beta = 0.9
rho = data.loc[9, 'rho']
nu = data.loc[9, 'nu']
sigmasabr = SABR(F, 0.2, 5, alpha, beta, rho, nu)

### Part 1 
> Value the payoff of $$CMS10y^{\frac{1}{p}}-0.04^{\frac{1}{q}}$$ The CMS tenor 5 years starts, tenor is 5 years.

In [6]:
# def integral1(x,N,m,F,disc,sigma,T):
#     h = hff(x, N, m)
#     Vrec=Black76Put(F, x, disc, sigma, T)
#     return h*Vrec

# def integral2(x,N,m,F,disc,sigma,T):
#     h = hff(x, N, m)
#     Vpay=Black76Call(F, x, disc, sigma, T)
#     return h*Vpay

In [8]:
# def Black76Call(S, K,disc, sigma, T):
#     d1 = (np.log(S/K)+sigma**2/2*T) / (sigma*np.sqrt(T))
#     d2 = d1 - sigma*np.sqrt(T)
#     return disc*(S*norm.cdf(d1) - K*norm.cdf(d2))

def Black76Put(S, K, disc,sigma, T):
    d1 = (np.log(S/K)+sigma**2/2*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return disc*(K*norm.cdf(-d2) - S*norm.cdf(-d1))

In [9]:
Black76Put(100, 80, 0.03, 2.5,3)

2.3185575216968592

In [1]:
p = 4
q=2
term1 = F
term3 = quad(lambda x: integral1(x, 10, 2, F, IRR(F, 10, 2), sigmasabr, 5), 0, F)
term4 = quad(lambda x: integral2(x, 10, 2, F, IRR(F, 10, 2), sigmasabr, 5), F, np.inf)
PVoption = ((term1 + np.sum(term3 + term4))**(1/p) - (0.04)**(1/q))*D

NameError: name 'F' is not defined

In [2]:
PVoption

NameError: name 'PVoption' is not defined

### Part 2
>Value the payoff of $$(CMS10y^{\frac{1}{p}}-0.04^{\frac{1}{q}})^{+}$$

In [3]:
L = 0.0016

In [4]:
term5 = hf(0.0016, 10, 2) * Black76Call(F, 0.0016, IRR(F, 10, 2), sigmasabr, 5)
term6 = quad(lambda x: integral2(x, 10, 2, F, IRR(F, 10, 2), sigmasabr, 5), 0.0016, np.inf)
PVoption2 = (term5 + np.sum(term6))+ D*(F-L)

NameError: name 'hf' is not defined

In [5]:
PVoption2

NameError: name 'PVoption2' is not defined